In [36]:
! pip install joblib

  Using cached joblib-0.11-py2.py3-none-any.whl
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import log_loss
import numpy as np
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV, GridSearchCV

from gc import collect

import time
import joblib

In [40]:
TARGET = ['identity_hate', 'insult', 'obscene','severe_toxic', 'threat', 'toxic']

In [5]:
data =  pd.read_csv('data/train.csv', sep=',', index_col='id')
data['train'] = 1
data = pd.concat([data, pd.read_csv('data/test.csv', sep=',', index_col='id')], axis=0)
data.train.fillna(0, inplace=True)
print(data.shape)
data.head()

(312735, 8)


,comment_text,identity_hate,insult,obscene,severe_toxic,threat,toxic,train
id,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0.0,0.0,0.0,0.0,0.0,0.0,1.0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0.0,0.0,0.0,0.0,0.0,0.0,1.0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [27]:
pipe = Pipeline([('tfidf', TfidfVectorizer()), ('lg', LogisticRegression(random_state=234))])

In [7]:
space = {
 'lg__C': [.0001, .001, .01, .1, 1.0, 10, 100, 1000],
 'lg__class_weight': [{1:2, 0:1},{1:3, 0:1}, {1:4, 0:1}, 'balanced', None],
 'lg__fit_intercept': [True,False],
 'lg__max_iter': [100, 200, 300, 500, 1000],
 'lg__penalty': ['l2', 'l1'],
 'lg__random_state': [234],
 'lg__tol': [0.0001, .001, .01],
 'lg__warm_start': [False,True],
 #'tfidf__analyzer': 'word',
'tfidf__binary': [False,True],
 'tfidf__decode_error': ['ignore'],
 #'tfidf__input': 'content',
 'tfidf__lowercase': [False,True],
 'tfidf__max_df': list(np.linspace(0.8, 1.0, 20)),
# 'tfidf__max_features': [10000, None], may be it's false
 'tfidf__min_df': [1, 5, 10, 20, 30, 50, 100],
 'tfidf__ngram_range': [(1, 1), (2, 1), (2, 2)], #(3, 3), (4, 1) 3, 1), , (3, 2)
 'tfidf__norm': ['l2','l1', None],
 'tfidf__smooth_idf': [False,True],
 #'tfidf__stop_words': None,
 #'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': [False,True],
 #'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__use_idf': [False,True]
 #'tfidf__vocabulary': None
        }

In [ ]:
np.random.seed(234)

for i, tar in enumerate(TARGET):
    flag_app = True
    estimator = pipe
    for key in space.keys():
        if 'lg' not in key:
            print(tar, key, time.strftime('%H:%M'))
            grid = GridSearchCV(estimator, param_grid={k:v for k, v in space.items() if k == key}
                                , scoring='neg_log_loss', 
                           cv = 5, verbose=0, n_jobs = 5)
            grid.fit(data.loc[data.train==1, 'comment_text'], data.loc[data.train==1, tar].values)
            estimator = grid.best_estimator_
            if flag_app:
                pd.DataFrame(grid.cv_results_).to_csv('metrics/grid_search_'+tar+'_.csv', sep=';', index=False)
                flag_app = False
            else:
                pd.DataFrame(grid.cv_results_).to_csv('metrics/grid_search_'+tar+'_.csv', mode='a', sep=';', index=False)
    joblib.dump(estimator, 'models/best_model_tfidf_'+tar+'.pkl')

identity_hate tfidf__max_df 19:49
identity_hate tfidf__smooth_idf 20:02
identity_hate tfidf__binary 20:04
identity_hate tfidf__decode_error 20:05
identity_hate tfidf__use_idf 20:06
identity_hate tfidf__norm 20:08


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


identity_hate tfidf__min_df 20:11
identity_hate tfidf__sublinear_tf 20:16
identity_hate tfidf__ngram_range 20:17
identity_hate tfidf__lowercase 20:19
insult tfidf__max_df 20:20
insult tfidf__smooth_idf 20:35
insult tfidf__binary 20:37
insult tfidf__decode_error 20:38
insult tfidf__use_idf 20:39
insult tfidf__norm 20:41


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


insult tfidf__min_df 20:45
insult tfidf__sublinear_tf 20:49
insult tfidf__ngram_range 20:50
insult tfidf__lowercase 20:53
obscene tfidf__max_df 20:54
obscene tfidf__smooth_idf 21:09
obscene tfidf__binary 21:10
obscene tfidf__decode_error 21:12
obscene tfidf__use_idf 21:13
obscene tfidf__norm 21:14
obscene tfidf__min_df 21:18
obscene tfidf__sublinear_tf 21:22
obscene tfidf__ngram_range 21:24
obscene tfidf__lowercase 21:26


In [ ]:
np.random.seed(234)

for i, tar in enumerate(TARGET):
    estimator = joblib.load('models/best_model_tfidf_'+tar+'.pkl')
    for key in space.keys():
        if 'lg' in key:
            print(tar, key, time.strftime('%H:%M'))
            grid = GridSearchCV(estimator, param_grid={k:v for k, v in space.items() if k == key}
                                , scoring='neg_log_loss', 
                           cv = 5, verbose=0, n_jobs = 5)
            grid.fit(data.loc[data.train==1, 'comment_text'], data.loc[data.train==1, tar].values)
            estimator = grid.best_estimator_
            if flag_app:
                pd.DataFrame(grid.cv_results_).to_csv('metrics/grid_search_'+tar+'_.csv', sep=';', index=False)
                flag_app = False
            else:
                pd.DataFrame(grid.cv_results_).to_csv('metrics/grid_search_'+tar+'_.csv', mode='a', sep=';', index=False)
    joblib.dump(estimator, 'models/best_model_lr_'+tar+'.pkl')

In [45]:
submission = pd.read_csv('data/sample_submission.csv', sep=',', index_col='id')
for i, tar in enumerate(TARGET):
    estimator = joblib.load('models/best_model_lr_'+tar+'.pkl')
    estimator.fit(data.loc[data.train==1, 'comment_text'], data.loc[data.train==1, tar].values)
    submission[tar] = estimator.predict_proba(data.loc[data.train!=1, 'comment_text'])[:,1]
submission.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,0.999968,0.268023,0.999538,0.212522,0.971607,0.280666
0000247867823ef7,0.000655,0.001599,0.000212,0.000040,0.004851,0.000669
00013b17ad220c46,0.004070,0.001584,0.002914,0.000037,0.006084,0.000730
00017563c3f7919a,0.001320,0.002485,0.002222,0.000271,0.003697,0.000204
00017695ad8997eb,0.007529,0.003709,0.001754,0.001016,0.005635,0.000638


In [46]:
submission.to_csv('prediction/sub_fris_grid_search.csv', sep=',')

In [47]:
! kg config -c jigsaw-toxic-comment-classification-challenge -u cerdgio86@gmail.com -p 68918082

! kg submit 'prediction/sub_fris_grid_search.csv' -m "____"

0.076
